# Problème de Coloration de Graphe

Le **problème de coloration de graphe** peut être décrit mathématiquement de la manière suivante :

Soit G = (V, E) un graphe non orienté, où :

* V est l’ensemble des sommets (ou nœuds) du graphe.
* E est l’ensemble des arêtes, où chaque arête e = (u, v) représente une connexion entre deux sommets u et v.

---

### Objectif

L’objectif est d’attribuer une couleur à chaque sommet de V de telle sorte que deux sommets adjacents ne partagent jamais la même couleur. Le **nombre chromatique** χ(G) du graphe G est le nombre minimum de couleurs nécessaires pour colorier les sommets de cette manière. Formellement :

χ(G) = min { k : il existe une fonction f : V → {1, 2, ..., k} telle que pour tout (u, v) ∈ E, f(u) ≠ f(v) }

Où :

* f : V → {1, 2, ..., k} est une fonction de coloration qui attribue une couleur (représentée par un entier) à chaque sommet.
* La contrainte f(u) ≠ f(v) garantit que les sommets adjacents reçoivent des couleurs différentes.

---

### Complexité Computationnelle

Déterminer le nombre chromatique χ(G) d’un graphe est un problème bien connu comme étant **NP-difficile**. Cela signifie qu’il n’existe, en général, aucun algorithme efficace pour trouver le nombre chromatique exact d’un graphe quelconque en temps polynomial, sauf si P = NP.

Ainsi, trouver la solution optimale (c’est-à-dire le nombre chromatique exact) est coûteux en termes de calcul, notamment pour les grands graphes. Cela rend les solutions exactes peu pratiques dans les applications réelles où les graphes peuvent comporter des milliers voire des millions de sommets et d’arêtes.

---

### Solutions Heuristiques

Compte tenu de la difficulté à résoudre exactement le problème de coloration de graphe, on utilise couramment des **heuristiques** pour obtenir des solutions approximatives. Ces heuristiques visent à trouver une solution proche du nombre chromatique optimal, mais en un temps beaucoup plus court.

---

### Approche Hyper-Heuristique

L’hyper-heuristique implémentée dans ce notebook est un cadre d’optimisation de haut niveau qui fait évoluer une population d’heuristiques de bas niveau afin de résoudre efficacement le problème de coloration de graphe. Contrairement aux heuristiques traditionnelles qui opèrent directement sur l’espace des solutions (c’est-à-dire les attributions de couleurs), une hyper-heuristique opère sur l’espace des heuristiques, en sélectionnant et combinant celles-ci pour générer de bonnes solutions. L’approche mise en œuvre comporte les éléments clés suivants :

* **Heuristique de bas niveau (`ColoringHeuristic`)** : Chaque heuristique de la population est une instance de la classe `ColoringHeuristic`, qui combine une stratégie de coloration gloutonne avec une recherche tabou pour l’optimisation locale. L’heuristique utilise une combinaison pondérée de trois critères pour ordonner les sommets à colorier :

  * **Degré** : Le nombre de voisins d’un sommet, en donnant la priorité aux sommets de degré élevé.
  * **Saturation** : Le nombre de couleurs différentes utilisées par les voisins d’un sommet, pour traiter en priorité les sommets les plus contraints.
  * **Conflits** : Le nombre de conflits (sommets adjacents ayant la même couleur), en ciblant les sommets impliqués dans des conflits à corriger.

  Les poids associés à ces critères (`weightDegree`, `weightSaturation`, `weightConflicts`) sont des paramètres évolués par l’hyper-heuristique, ce qui lui permet d’adapter sa stratégie d’ordonnancement des sommets.

  L’heuristique inclut également une recherche locale basée sur la recherche tabou, contrôlée par des paramètres comme `localSearchProb` (probabilité d’appliquer la recherche locale), `tabuTenure` (durée d’interdiction d’un mouvement), `maxIterations` (nombre d’itérations de recherche locale), et `perturbationRate` (taux d’introduction de modifications aléatoires pour échapper aux optima locaux).

* **Algorithme génétique (`HyperHeuristic`)** : L’hyper-heuristique utilise un algorithme génétique pour faire évoluer une population d’instances `ColoringHeuristic` sur plusieurs générations. Les opérations clés sont :

  * **Initialisation** : Création d’une population d’heuristiques avec des paramètres aléatoires.
  * **Sélection** : La sélection par tournoi choisit les parents selon leur performance.
  * **Croisement** : De nouvelles heuristiques sont créées en combinant les paramètres de deux parents, chaque paramètre étant choisi aléatoirement chez l’un des deux.
  * **Mutation** : Les paramètres sont modifiés aléatoirement avec un certain taux de mutation, en ajustant les poids, la durée tabou, le nombre d’itérations ou le taux de perturbation dans des plages définies.
  * **Élitisme** : Les 10 % meilleurs heuristiques sont conservés à chaque génération pour préserver les meilleures solutions.

* **Recherche Locale (Recherche Tabou)** : La `ColoringHeuristic` applique la recherche tabou pour améliorer les colorations initiales en changeant les couleurs des sommets de manière itérative afin de réduire les conflits et le nombre de couleurs utilisées. Elle maintient une liste tabou pour éviter les cycles, utilise des critères d’aspiration pour permettre des mouvements prometteurs et emploie des stratégies de diversification (modérées et fortes) pour sortir des stagnations.

* **Lecture de Graphe** : L’hyper-heuristique lit les graphes au format DIMACS, un standard utilisé dans les benchmarks de coloration de graphe, et évalue les heuristiques sur ces graphes.

L’objectif de l’hyper-heuristique est de découvrir une heuristique de coloration robuste qui généralise bien à différents graphes en ajustant ses paramètres via l’évolution. Le notebook illustre cela en s’entraînant sur un graphe DIMACS et en testant la meilleure heuristique trouvée, en rapportant le nombre de couleurs utilisées, la validité de la coloration et les temps d’exécution.



In [ ]:
import numpy as np
import random
import time
from collections import defaultdict

# Classe **ColoringHeuristic**

La classe `ColoringHeuristic` définit une heuristique individuelle pour la **coloration de graphe**. Elle comprend des méthodes pour :

* initialiser les paramètres,
* effectuer des opérations de **croisement** (crossover) et de **mutation**,
* déterminer l’ordre des sommets à colorier,
* colorier le graphe,
* et appliquer une **recherche locale** (recherche tabou).

La classe inclut également des méthodes utilitaires permettant :

* d’évaluer la qualité des colorations,
* et de gérer les conflits entre sommets adjacents.

In [ ]:
class ColoringHeuristic:
    def __init__(self, params=None):
        # Si des paramètres sont fournis, les utiliser, sinon générer aléatoirement
        self.params = params or {
            # Poids pour différentes stratégies de sélection de nœuds
            'weightDegree': random.random(),
            'weightSaturation': random.random(),
            'weightConflicts': random.random(),

            # Paramètres pour la recherche locale
            'localSearchProb': 0.9 + random.random() * 0.1,  # Élevé pour favoriser la recherche locale
            'tabuTenure': random.randint(5, 15),
            'maxIterations': random.randint(100, 500),

            # Paramètres pour la perturbation
            'perturbationRate': random.random() * 0.2
        }

        self.fitness = 0

    # Croisement avec une autre heuristique
    def crossover(self, other):
        child_params = {}

        # Pour chaque paramètre, prendre aléatoirement celui d'un parent ou faire une moyenne
        for key in self.params:
            if random.random() < 0.5:
                child_params[key] = self.params[key]
            else:
                child_params[key] = other.params[key]

        return ColoringHeuristic(child_params)

    # Mutation de l'heuristique
    def mutate(self, mutation_rate):
        new_params = self.params.copy()

        for key in new_params:
            if random.random() < mutation_rate:
                # Différentes stratégies de mutation selon le type de paramètre
                if key == 'tabuTenure':
                    new_params[key] = random.randint(5, 15)
                elif key == 'maxIterations':
                    new_params[key] = random.randint(100, 500)
                elif key == 'localSearchProb':
                    # Garder une probabilité élevée pour la recherche locale
                    new_params[key] = 0.9 + random.random() * 0.1
                elif key.startswith('weight'):
                    new_params[key] = random.random()
                else:
                    # Perturber légèrement la valeur actuelle
                    new_params[key] = max(0, min(1, new_params[key] + (random.random() * 0.4 - 0.2)))

        return ColoringHeuristic(new_params)

    # Déterminer l'ordre dans lequel colorer les nœuds
    def _determine_node_order(self, adjacency_matrix, colors=None):
        num_nodes = len(adjacency_matrix)
        node_scores = []

        # Si aucune coloration n'est fournie, initialiser avec -1 (non coloré)
        if colors is None:
            colors = [-1] * num_nodes

        for node in range(num_nodes):
            # 1. Calculer le degré (nombre de voisins)
            degree = sum(adjacency_matrix[node])

            # 2. Calculer la saturation (nombre de couleurs différentes utilisées par les voisins)
            saturation = 0
            if any(colors[i] != -1 for i in range(num_nodes)):  # Si des nœuds sont déjà colorés
                neighbor_colors = set()
                for neighbor in range(num_nodes):
                    if adjacency_matrix[node][neighbor] == 1 and colors[neighbor] != -1:
                        neighbor_colors.add(colors[neighbor])
                saturation = len(neighbor_colors)

            # 3. Calculer les conflits potentiels
            conflicts = 0
            if any(colors[i] != -1 for i in range(num_nodes)):  # Si des nœuds sont déjà colorés
                for neighbor in range(num_nodes):
                    if adjacency_matrix[node][neighbor] == 1 and colors[neighbor] == colors[node] and colors[node] != -1:
                        conflicts += 1

            # Calculer le score en fonction des poids de l'heuristique
            score = (degree * self.params['weightDegree'] +
                     saturation * self.params['weightSaturation'] +
                     conflicts * self.params['weightConflicts'])

            node_scores.append({'node': node, 'score': score, 'degree': degree})

        # Trier les nœuds par score décroissant
        node_scores.sort(key=lambda x: (x['score'], x['degree']), reverse=True)

        return [item['node'] for item in node_scores]

    # Appliquer l'heuristique pour colorer un graphe
    def color_graph(self, adjacency_matrix):
        num_nodes = len(adjacency_matrix)
        colors = [-1] * num_nodes

        # Coloration progressive
        for i in range(num_nodes):
            # Recalculer l'ordre des nœuds à chaque étape en tenant compte de la coloration partielle
            node_order = self._determine_node_order(adjacency_matrix, colors)

            # Trouver le premier nœud non coloré dans l'ordre
            node = next((n for n in node_order if colors[n] == -1), None)
            if node is None:
                break

            used_colors = set()
            # Trouver les couleurs utilisées par les voisins
            for neighbor in range(num_nodes):
                if adjacency_matrix[node][neighbor] == 1 and colors[neighbor] != -1:
                    used_colors.add(colors[neighbor])

            # Trouver la plus petite couleur disponible
            color = 0
            while color in used_colors:
                color += 1

            colors[node] = color

        # Appliquer la recherche locale si nécessaire
        if random.random() < self.params['localSearchProb']:
            colors = self._local_search(adjacency_matrix, colors)

        return colors

    # Recherche locale basée sur la recherche tabou
    def _local_search(self, adjacency_matrix, colors):
        """
        Effectue une recherche locale basée sur la recherche tabou pour améliorer une coloration initiale,
        en minimisant les conflits et le nombre de couleurs.

        Args:
            adjacency_matrix: Matrice d'adjacence du graphe
            colors: Liste des couleurs initiales assignées aux nœuds

        Returns:
            Liste des couleurs après optimisation par recherche locale
        """
        num_nodes = len(adjacency_matrix)
        max_iterations = self.params['maxIterations']
        tabu_tenure = self.params['tabuTenure']
        perturbation_rate = self.params['perturbationRate']
        aspiration_factor = 0.9  # Facteur pour le critère d'aspiration
        diversification_threshold = 100  # Seuil pour la diversification

        # Initialiser la solution courante à partir de la coloration fournie
        current_coloring = colors.copy()
        num_colors = max(current_coloring) + 1 if current_coloring else 0
        conflicts = len(self._find_conflicts(adjacency_matrix, current_coloring))
        current_objective = self._calculate_objective(num_colors, conflicts)

        # Initialiser la meilleure solution
        best_coloring = current_coloring.copy()
        best_objective = current_objective
        best_num_colors = num_colors
        best_conflicts = conflicts

        # Initialiser les structures de données pour la recherche tabou
        tabu_list = {}  # (vertex, color) -> expiration_iteration
        frequency = defaultdict(int)  # (vertex, color) -> fréquence d'utilisation
        stagnation_counter = 0
        last_improvement = 0

        for iteration in range(max_iterations):
            # Identifier les nœuds candidats
            if conflicts > 0:
                # Si des conflits existent, se concentrer sur les nœuds en conflit
                candidates = self._find_conflicts(adjacency_matrix, current_coloring)
            else:
                # Sinon, essayer de réduire le nombre de couleurs
                max_color = max(current_coloring) if current_coloring else 0
                candidates = [i for i in range(num_nodes) if current_coloring[i] == max_color]
                if not candidates:
                    candidates = list(range(num_nodes))

            # Limiter le nombre de candidats pour l'efficacité
            if len(candidates) > 50:
                candidates = random.sample(candidates, 50)

            best_move = None
            best_move_objective = float('inf')

            # Évaluer tous les mouvements possibles
            for vertex in candidates:
                old_color = current_coloring[vertex]
                max_color = max(current_coloring) if current_coloring else 0

                # Déterminer les couleurs à essayer, avec perturbationRate pour ajouter une nouvelle couleur
                colors_to_try = set(range(max_color + (1 if random.random() < perturbation_rate else 0) + 1))
                if conflicts == 0:
                    colors_to_try = set(range(max_color))  # Exclure la couleur maximale pour réduire le nombre de couleurs
                colors_to_try.discard(old_color)

                for new_color in colors_to_try:
                    # Vérifier si le mouvement est tabou
                    is_tabu = tabu_list.get((vertex, new_color), 0) > iteration

                    # Simuler le mouvement
                    current_coloring[vertex] = new_color
                    new_num_colors = max(current_coloring) + 1 if current_coloring else 0
                    new_conflicts = len(self._find_conflicts(adjacency_matrix, current_coloring))
                    new_objective = self._calculate_objective(new_num_colors, new_conflicts)

                    # Critère d'aspiration
                    aspiration_condition = new_objective < best_objective * aspiration_factor

                    if (not is_tabu or aspiration_condition) and new_objective < best_move_objective:
                        best_move = (vertex, new_color)
                        best_move_objective = new_objective

                    # Annuler la simulation
                    current_coloring[vertex] = old_color

            # Si aucun mouvement n'est trouvé, appliquer une diversification
            if best_move is None:
                # Calculer le nombre de nœuds à perturber en fonction de perturbationRate
                num_perturbations = max(1, int(num_nodes * perturbation_rate))
                for _ in range(num_perturbations):
                    v = random.choice(range(num_nodes))
                    old_color = current_coloring[v]
                    max_color = max(current_coloring) if current_coloring else 0
                    new_color = random.randint(0, max_color + (1 if random.random() < perturbation_rate else 0))
                    while new_color == old_color:
                        new_color = random.randint(0, max_color + (1 if random.random() < perturbation_rate else 0))
                    current_coloring[v] = new_color

                # Réévaluer après diversification
                num_colors = max(current_coloring) + 1 if current_coloring else 0
                conflicts = len(self._find_conflicts(adjacency_matrix, current_coloring))
                current_objective = self._calculate_objective(num_colors, conflicts)
                stagnation_counter = 0
                continue

            # Appliquer le meilleur mouvement
            vertex, new_color = best_move
            old_color = current_coloring[vertex]
            current_coloring[vertex] = new_color

            # Mettre à jour les valeurs courantes
            num_colors = max(current_coloring) + 1 if current_coloring else 0
            conflicts = len(self._find_conflicts(adjacency_matrix, current_coloring))
            current_objective = self._calculate_objective(num_colors, conflicts)

            # Mettre à jour la liste tabou
            tabu_list[(vertex, old_color)] = iteration + tabu_tenure + random.randint(-2, 2)

            # Mettre à jour la fréquence
            frequency[(vertex, new_color)] += 1

            # Mettre à jour la meilleure solution si nécessaire
            if (conflicts < best_conflicts) or (conflicts == best_conflicts and num_colors < best_num_colors):
                best_coloring = current_coloring.copy()
                best_objective = current_objective
                best_num_colors = num_colors
                best_conflicts = conflicts
                last_improvement = iteration
                stagnation_counter = 0
            else:
                stagnation_counter += 1

            # Diversification si stagnation
            if stagnation_counter >= diversification_threshold:
                if stagnation_counter >= 2 * diversification_threshold:
                    # Forte diversification : réassigner les couleurs peu utilisées
                    num_perturbations = max(5, int(num_nodes * perturbation_rate))
                    for v in random.sample(range(num_nodes), num_perturbations):
                        colors_freq = [(color, frequency.get((v, color), 0)) for color in range(max(current_coloring) + 1)]
                        colors_freq.sort(key=lambda x: x[1])
                        if colors_freq:
                            low_freq_colors = [c for c, f in colors_freq[:min(3, len(colors_freq))]]
                            if low_freq_colors:
                                current_coloring[v] = random.choice(low_freq_colors)
                else:
                    # Diversification modérée : modifier aléatoirement quelques nœuds
                    num_perturbations = max(1, int(num_nodes * perturbation_rate))
                    for _ in range(num_perturbations):
                        v = random.choice(range(num_nodes))
                        max_color = max(current_coloring) if current_coloring else 0
                        current_coloring[v] = random.randint(0, max_color)

                # Réévaluer après diversification
                num_colors = max(current_coloring) + 1 if current_coloring else 0
                conflicts = len(self._find_conflicts(adjacency_matrix, current_coloring))
                current_objective = self._calculate_objective(num_colors, conflicts)
                stagnation_counter = 0

            # Arrêter si aucune amélioration n'est possible
            if conflicts == 0 and num_colors <= best_num_colors:
                break

        return best_coloring

    # Calculer la valeur objective
    def _calculate_objective(self, num_colors, conflicts):
        """
        Calcule la valeur objective en combinant le nombre de conflits et le nombre de couleurs.

        Args:
            num_colors: Nombre de couleurs utilisées
            conflicts: Nombre de conflits dans la coloration

        Returns:
            Valeur objective (plus petite est meilleure)
        """
        conflict_penalty = 1000 * conflicts  # Pénalité élevée pour les conflits
        color_penalty = num_colors  # Pénalité pour le nombre de couleurs
        return conflict_penalty + color_penalty

    # Trouver les nœuds en conflit
    def _find_conflicts(self, adjacency_matrix, colors):
        num_nodes = len(adjacency_matrix)
        conflicts = []

        for i in range(num_nodes):
            for j in range(i + 1, num_nodes):
                if adjacency_matrix[i][j] == 1 and colors[i] == colors[j]:
                    if i not in conflicts:
                        conflicts.append(i)
                    if j not in conflicts:
                        conflicts.append(j)

        return conflicts

    # Compter les conflits pour un nœud avec une couleur spécifique
    def _count_node_conflicts(self, node, color, adjacency_matrix, colors):
        num_nodes = len(adjacency_matrix)
        conflicts = 0

        for neighbor in range(num_nodes):
            if adjacency_matrix[node][neighbor] == 1 and colors[neighbor] == color:
                conflicts += 1

        return conflicts

    # Évaluer la qualité de la coloration
    def evaluate_coloring(self, adjacency_matrix, colors):
        # Nombre de couleurs utilisées
        num_colors = max(colors) + 1 if colors else 0

        # Nombre de conflits
        conflicts = len(self._find_conflicts(adjacency_matrix, colors))

        # Vérifier si la coloration est valide
        is_valid = conflicts == 0

        return {
            'num_colors': num_colors,
            'conflicts': conflicts,
            'is_valid': is_valid
        }

# Classe **HyperHeuristic**

La classe `HyperHeuristic` gère une population d’instances de la classe `ColoringHeuristic`, qu’elle fait évoluer au fil des générations à l’aide de mécanismes de **sélection**, de **croisement** et de **mutation**, dans le but de trouver une heuristique efficace pour la **coloration de graphe**.

In [ ]:
class HyperHeuristic:
    def __init__(self, population_size=20, generations=50):
        self.population_size = population_size
        self.generations = generations
        self.population = []

        # Initialiser la population
        for i in range(population_size):
            self.population.append(ColoringHeuristic())

    # Sélectionner des parents par tournoi
    def select_parents(self):
        parents = []

        for i in range(self.population_size):
            # Sélection par tournoi
            tournament_size = 3
            tournament = []

            for j in range(tournament_size):
                random_index = random.randint(0, len(self.population) - 1)
                tournament.append(self.population[random_index])

            tournament.sort(key=lambda x: x.fitness, reverse=True)
            parents.append(tournament[0])

        return parents

    def create_new_generation(self):
        new_population = []

        # 1. Élitisme: conserver directement les 10% meilleurs
        elite_count = max(1, int(self.population_size * 0.1))
        for i in range(elite_count):
            new_population.append(self.population[i])

        # 2. Créer une population temporaire avec les 90% restants
        remaining_population = self.population[elite_count:]
        remaining_count = len(remaining_population)

        # 3. Sélectionner des parents uniquement parmi les 90% restants
        parents = []
        for i in range(remaining_count):
            # Sélection par tournoi
            tournament_size = 3
            tournament = []

            for j in range(tournament_size):
                random_index = random.randint(0, remaining_count - 1)
                tournament.append(remaining_population[random_index])

            tournament.sort(key=lambda x: x.fitness, reverse=True)
            parents.append(tournament[0])

        # 4. Appliquer le croisement sur les 90% restants
        crossover_rate = 0.7  # 70% des individus restants subiront un croisement
        crossover_count = int(remaining_count * crossover_rate)

        # Créer des individus par croisement
        for i in range(crossover_count):
            parent1 = parents[random.randint(0, len(parents) - 1)]
            parent2 = parents[random.randint(0, len(parents) - 1)]

            child = parent1.crossover(parent2)
            new_population.append(child)

        # 5. Compléter la population avec des copies des parents
        while len(new_population) < self.population_size:
            parent = parents[random.randint(0, len(parents) - 1)]
            new_population.append(ColoringHeuristic(parent.params.copy()))

        self.population = new_population

        # 6. Appliquer la mutation (en évitant les élites)
        self.apply_mutation()

    # Appliquer la mutation
    def apply_mutation(self):
        # Taux de mutation externe (pourcentage d'individus qui subiront une mutation)
        external_mutation_rate = 0.3  # 30% des individus seront mutés

        # Taux de mutation interne (probabilité de mutation pour chaque paramètre)
        internal_mutation_rate = 0.2  # 20% de chance de muter chaque paramètre

        # Nombre d'individus à muter
        mutation_count = int(self.population_size * external_mutation_rate)

        # Sélectionner aléatoirement des individus à muter (en évitant les élites)
        elite_count = max(1, int(self.population_size * 0.1))
        indices_to_mutate = random.sample(range(elite_count, self.population_size), mutation_count)

        # Appliquer la mutation
        for idx in indices_to_mutate:
            self.population[idx] = self.population[idx].mutate(internal_mutation_rate)

# Fonctions Utilitaires

Ces fonctions utilitaires accomplissent des tâches telles que :

* la **validation d'une coloration** (vérification que deux sommets adjacents n'ont pas la même couleur),
* et la **lecture d’un graphe au format DIMACS**.

In [ ]:
def is_valid_coloring(adjacency_matrix, colors):
    num_nodes = len(adjacency_matrix)

    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if adjacency_matrix[i][j] == 1 and colors[i] == colors[j]:
                return False

    return True

# Fonction pour lire un graphe au format DIMACS
def read_dimacs_graph(file_path):
    """Lit un graphe au format DIMACS et retourne sa matrice d'adjacence."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    edges = []
    num_nodes = 0

    for line in lines:
        line = line.strip()
        if line.startswith('p'):  # Problem line
            parts = line.split()
            num_nodes = int(parts[2])
        elif line.startswith('e'):  # Edge line
            parts = line.split()
            # DIMACS vertices are 1-indexed, convert to 0-indexed
            node1, node2 = int(parts[1]) - 1, int(parts[2]) - 1
            edges.append((node1, node2))

    # Create the adjacency matrix
    adjacency_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    # Fill the adjacency matrix
    for node1, node2 in edges:
        adjacency_matrix[node1, node2] = 1
        adjacency_matrix[node2, node1] = 1  # Undirected graph

    return adjacency_matrix

# Exécution de l’Hyper-Heuristique

La fonction `run_hyper_heuristic_on_dimacs` exécute l’hyper-heuristique sur un fichier de graphe au **format DIMACS**, en faisant évoluer une population d’heuristiques afin de trouver une **stratégie de coloration efficace**.

In [ ]:
def run_hyper_heuristic_on_dimacs(file_path, population_size=20, generations=50):
    """
    Exécute l'hyper-heuristique directement sur un fichier DIMACS spécifique.
    Retourne la meilleure heuristique et le temps d'entraînement.
    """
    # Lire le graphe DIMACS
    adjacency_matrix = read_dimacs_graph(file_path)

    # Créer l'hyper-heuristique
    hyper_heuristic = HyperHeuristic(population_size, generations)

    print(f"Démarrage de l'hyper-heuristique sur {file_path}...")

    # Mesurer le temps d'entraînement
    training_start_time = time.time()

    # Exécuter l'hyper-heuristique sur cette instance spécifique
    for gen in range(generations):
        # Évaluer la population sur cette instance spécifique
        for heuristic in hyper_heuristic.population:
            colors = heuristic.color_graph(adjacency_matrix)
            result = heuristic.evaluate_coloring(adjacency_matrix, colors)

            # La fitness est basée sur la validité et le nombre de couleurs
            if result['is_valid']:
                heuristic.fitness = 1 / result['num_colors']
            else:
                heuristic.fitness = 1 / (1000 + result['conflicts'] * 100 + result['num_colors'])

        # Trier la population par fitness décroissante
        hyper_heuristic.population.sort(key=lambda x: x.fitness, reverse=True)

        best_heuristic = hyper_heuristic.population[0]
        best_colors = best_heuristic.color_graph(adjacency_matrix)
        best_result = best_heuristic.evaluate_coloring(adjacency_matrix, best_colors)

        print(f"Génération {gen + 1}/{generations} - Meilleure fitness: {best_heuristic.fitness:.4f}")
        print(f"  Nombre de couleurs: {best_result['num_colors']}, Valide: {'Oui' if best_result['is_valid'] else 'Non'}")

        if gen < generations - 1:
            hyper_heuristic.create_new_generation()

    # Calculer le temps d'entraînement total
    training_time = time.time() - training_start_time
    print(f"Temps d'entraînement total: {training_time:.2f} secondes")

    # Retourner la meilleure heuristique et le temps d'entraînement
    return hyper_heuristic.population[0], training_time

# Test de la Meilleure Heuristique

La fonction `test_best_heuristic` teste la **meilleure heuristique** trouvée par l’hyper-heuristique sur un fichier de graphe au format **DIMACS**, et affiche les résultats, y compris :

* le **temps d’exécution**,
* et la **validité de la coloration**.

In [ ]:
def test_best_heuristic(heuristic, file_path):
    """
    Teste la meilleure heuristique sur un fichier DIMACS et mesure son temps d'exécution.
    """
    # Lire le graphe DIMACS
    adjacency_matrix = read_dimacs_graph(file_path)

    print(f"\nTest de la meilleure heuristique sur {file_path}...")

    # Mesurer le temps d'exécution de la meilleure heuristique
    execution_start_time = time.time()

    # Colorer le graphe avec la meilleure heuristique
    colors = heuristic.color_graph(adjacency_matrix)

    # Calculer le temps d'exécution
    execution_time = time.time() - execution_start_time

    # Évaluer la coloration
    result = heuristic.evaluate_coloring(adjacency_matrix, colors)

    print(f"Résultats:")
    print(f"Nombre de couleurs utilisées: {result['num_colors']}")
    print(f"Coloration valide: {'Oui' if result['is_valid'] else 'Non'}")
    print(f"Temps d'exécution: {execution_time:.6f} secondes")

    return {
        'colors': colors,
        'num_colors': result['num_colors'],
        'is_valid': result['is_valid'],
        'execution_time': execution_time
    }

# Exécution de l’Hyper-Heuristique

Cette cellule exécute l’**hyper-heuristique** sur un fichier DIMACS d’exemple, puis teste la **meilleure heuristique trouvée**.

In [ ]:
if __name__ == "__main__":
    # Chemin vers votre fichier DIMACS
    file_path = "/content/dsjc250.5.col"  # Remplacez par le chemin réel de votre fichier

    # 1. Entraîner l'hyper-heuristique sur cette instance spécifique
    best_heuristic, training_time = run_hyper_heuristic_on_dimacs(
        file_path,
        population_size=30,  # Taille de la population
        generations=10      # Nombre de générations
    )

    # 2. Tester la meilleure heuristique trouvée sur la même instance
    result = test_best_heuristic(best_heuristic, file_path)

    # 3. Afficher les résultats
    print("\nRésultats finaux pour", file_path)
    print(f"Nombre de couleurs utilisées: {result['num_colors']}")
    print(f"Coloration valide: {'Oui' if result['is_valid'] else 'Non'}")
    print(f"Temps d'entraînement: {training_time:.2f} secondes")
    print(f"Temps d'exécution: {result['execution_time']:.6f} secondes")

    # 4. Afficher les paramètres de la meilleure heuristique
    print("\nParamètres de la meilleure heuristique:")
    for key, value in best_heuristic.params.items():
        if isinstance(value, float):
            print(f"{key}: {value:.4f}")
        else:
            print(f"{key}: {value}")

Démarrage de l'hyper-heuristique sur /content/dsjc250.5.col...
Génération 1/10 - Meilleure fitness: 0.0270
  Nombre de couleurs: 37, Valide: Oui
Génération 2/10 - Meilleure fitness: 0.0270
  Nombre de couleurs: 37, Valide: Oui
Génération 3/10 - Meilleure fitness: 0.0270
  Nombre de couleurs: 37, Valide: Oui
Génération 4/10 - Meilleure fitness: 0.0270
  Nombre de couleurs: 37, Valide: Oui
Génération 5/10 - Meilleure fitness: 0.0270
  Nombre de couleurs: 37, Valide: Oui
Génération 6/10 - Meilleure fitness: 0.0270
  Nombre de couleurs: 37, Valide: Oui
Génération 7/10 - Meilleure fitness: 0.0270
  Nombre de couleurs: 38, Valide: Oui
Génération 8/10 - Meilleure fitness: 0.0278
  Nombre de couleurs: 37, Valide: Oui
Génération 9/10 - Meilleure fitness: 0.0270
  Nombre de couleurs: 37, Valide: Oui
Génération 10/10 - Meilleure fitness: 0.0270
  Nombre de couleurs: 36, Valide: Oui
Temps d'entraînement total: 4843.14 secondes

Test de la meilleure heuristique sur /content/dsjc250.5.col...
Résulta